# 🎓 Knowledge Distillation From Scratch: Teacher to Student

[!["Open In Colab"](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/model-size-reduction/blob/main/distillation_demo.ipynb)

## 📖 The Theory: Transferring dark knowledge

Knowledge Distillation (KD) is a compression method where a compact model (**Student**) learns to mimic a large model (**Teacher**). Hinton et al. (2015) introduced the concept of "Dark Knowledge"—the relative probabilities of incorrect classes assigned by the teacher, which reveal the teacher's internal structure.

### The Loss Function
The student is trained to minimize a weighted sum of two losses:
1.  **Distillation Loss**: KL-Divergence between the teacher's and student's soft-targets.
2.  **Student Loss**: Standard Cross-Entropy between the student's output and the ground truth (hard labels).

### Temperature (T)
We use a temperature $T$ to soften the probability distributions. A higher $T$ makes the "dark knowledge" (the small probabilities of incorrect classes) more visible.

$$q_i = \frac{\exp(z_i / T)}{\sum_j \exp(z_j / T)}$$

The distillation loss is then scaled by $T^2$ to keep the gradient magnitudes consistent when changing $T$.

---

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import GPT2LMHeadModel, GPT2Config, GPT2Tokenizer

def manual_distillation_loss(student_logits, teacher_logits, labels, T=2.0, alpha=0.5):
    """
    Implementation of the Distillation Loss from scratch.
    """
    # 1. Soft Targets (Teacher and Student)
    # We use log_softmax for the student and softmax for the teacher for KLDivLoss
    soft_targets = F.softmax(teacher_logits / T, dim=-1)
    soft_prob = F.log_softmax(student_logits / T, dim=-1)
    
    distillation_loss = nn.KLDivLoss(reduction="batchmean")(soft_prob, soft_targets) * (T * T)
    
    # 2. Standard Cross Entropy
    # student_logits: [batch, seq, vocab] -> [batch*seq, vocab]
    # labels: [batch, seq]
    student_loss = F.cross_entropy(student_logits.view(-1, student_logits.size(-1)), labels.view(-1))
    
    # 3. Combined Loss
    return alpha * distillation_loss + (1 - alpha) * student_loss

# 1. Setup Models
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
teacher = GPT2LMHeadModel.from_pretrained("gpt2").eval()

# Student: 2 layers, small hidden size
student_cfg = GPT2Config(n_layer=2, n_head=4, n_embd=256, vocab_size=tokenizer.vocab_size)
student = GPT2LMHeadModel(student_cfg)

print(f"Teacher Size: {sum(p.numel() for p in teacher.parameters())/1e6:.1f}M")
print(f"Student Size: {sum(p.numel() for p in student.parameters())/1e6:.1f}M")

## 🔄 The Distillation Loop
A manual step-by-step distillation training step.

In [ ]:
optimizer = torch.optim.Adam(student.parameters(), lr=1e-4)

inputs = tokenizer("The quick brown fox jumps over the lazy dog", return_tensors="pt")
labels = inputs["input_ids"]

# Step 1: Get Teacher Knowledge (no gradients here)
with torch.no_grad():
    teacher_out = teacher(**inputs)
    teacher_logits = teacher_out.logits

# Step 2: Student Forward Pass
student_out = student(**inputs)
student_logits = student_out.logits

# Step 3: Compute Loss & Update
loss = manual_distillation_loss(student_logits, teacher_logits, labels)
loss.backward()
optimizer.step()

print(f"Training step lost: {loss.item():.4f}")